#  价格预测特征工程 
1.  同一起飞日期历史价格
2.  同一提前天数历史价格
3.  季节特征

# 导入包

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot
import datetime
from datetime import datetime, date, timedelta
import time
import re # 正则表达式
from sklearn.preprocessing import MinMaxScaler

In [2]:
#航线维度数据处理后的文件
inputfile='../datasets/top500_airroute_preprocessed_1.csv'
outfile='../datasets/top500_airroute_feature_preprocessed_1.csv'

In [25]:
%time df= pd.read_csv(inputfile,parse_dates=['dep_date','search_date'])

CPU times: user 5.09 s, sys: 162 ms, total: 5.25 s
Wall time: 7.48 s


In [4]:
df.head()

,dep_date,search_date,pre_day,dep_city,arr_city,minprice,route,label
0,2016-01-01,2015-11-03,59,三亚,上海,NaN,三亚-上海,0.0
1,2016-01-01,2015-11-04,58,三亚,上海,NaN,三亚-上海,0.0
2,2016-01-01,2015-11-05,57,三亚,上海,NaN,三亚-上海,0.0
3,2016-01-01,2015-11-06,56,三亚,上海,NaN,三亚-上海,0.0
4,2016-01-01,2015-11-07,55,三亚,上海,NaN,三亚-上海,0.0


In [5]:
df.tail()

,dep_date,search_date,pre_day,dep_city,arr_city,minprice,route,label
2756995,2018-07-07,2018-07-03,4,乌鲁木齐,济南,1190.0,乌鲁木齐-济南,49.0
2756996,2018-07-07,2018-07-04,3,乌鲁木齐,济南,1190.0,乌鲁木齐-济南,49.0
2756997,2018-07-07,2018-07-05,2,乌鲁木齐,济南,1190.0,乌鲁木齐-济南,49.0
2756998,2018-07-07,2018-07-06,1,乌鲁木齐,济南,1190.0,乌鲁木齐-济南,49.0
2756999,2018-07-07,2018-07-07,0,乌鲁木齐,济南,1190.0,乌鲁木齐-济南,49.0


In [6]:
#数据验证
tmp=df.sort_values(by=['dep_date','search_date'])
len(tmp.dep_date.value_counts(ascending=True)==60) #按出发日期为准，提前天数记录有31条的数据860

919

### 特征工程处理总调用  
特征构造-相同起飞日期，历史15天每天搜索价格特征耗时：3609秒,空值数量：109240  
特征构造-相同提前天数，历史15天每天搜索价格特征耗时：3870秒,空值数量：7375  
 

In [28]:
# 生成季节性和历史性特征
%time res = feature_engineering(df)

/notebooks/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



特征构造-相同起飞日期，历史15天每天搜索价格特征耗时：54秒
特征构造-相同提前天数，历史15天每天搜索价格特征耗时：50秒,填充空值：120
特征构造--相同起飞日期价格统计量，耗时：51秒
特征构造--相同起飞日期价格统计量，耗时：50秒

特征构造-相同起飞日期，历史15天每天搜索价格特征耗时：53秒
特征构造-相同提前天数，历史15天每天搜索价格特征耗时：49秒,填充空值：120
特征构造--相同起飞日期价格统计量，耗时：50秒
特征构造--相同起飞日期价格统计量，耗时：51秒

特征构造-相同起飞日期，历史15天每天搜索价格特征耗时：54秒
特征构造-相同提前天数，历史15天每天搜索价格特征耗时：50秒,填充空值：120
特征构造--相同起飞日期价格统计量，耗时：51秒
特征构造--相同起飞日期价格统计量，耗时：50秒

特征构造-相同起飞日期，历史15天每天搜索价格特征耗时：53秒
特征构造-相同提前天数，历史15天每天搜索价格特征耗时：49秒,填充空值：120
特征构造--相同起飞日期价格统计量，耗时：50秒
特征构造--相同起飞日期价格统计量，耗时：50秒

特征构造-相同起飞日期，历史15天每天搜索价格特征耗时：53秒
特征构造-相同提前天数，历史15天每天搜索价格特征耗时：49秒,填充空值：120
特征构造--相同起飞日期价格统计量，耗时：51秒
特征构造--相同起飞日期价格统计量，耗时：50秒

特征构造-相同起飞日期，历史15天每天搜索价格特征耗时：53秒
特征构造-相同提前天数，历史15天每天搜索价格特征耗时：49秒,填充空值：120
特征构造--相同起飞日期价格统计量，耗时：50秒
特征构造--相同起飞日期价格统计量，耗时：50秒

特征构造-相同起飞日期，历史15天每天搜索价格特征耗时：53秒
特征构造-相同提前天数，历史15天每天搜索价格特征耗时：49秒,填充空值：120
特征构造--相同起飞日期价格统计量，耗时：50秒
特征构造--相同起飞日期价格统计量，耗时：50秒

特征构造-相同起飞日期，历史15天每天搜索价格特征耗时：53秒
特征构造-相同提前天数，历史15天每天搜索价格特征耗时：49秒,填充空值：120
特征构造--相同起飞日期价格统计量，耗时：50秒
特征构造--相同起飞日期价格统计量，耗时：50秒

特征构造-相同

In [29]:
%time res.to_csv(outfile,index=False)

# 生成特征工程的函数

In [24]:
def feature_engineering(df):
    route_list = route_split(df)
    res = pd.DataFrame()
    for tmp in route_list:
        data=history_15d_by_dep_optimize(tmp,columns_15d_by_dep)
        data=history_15d_by_preday_optimize(data,columns_15d_by_preday)
        data=seasonal_feature_engineering(data)
        data=stats_by_dep_feature(data,columns_by_dep_stats,columns_15d_by_dep)
        data=stats_by_preday_feature(data,columns_by_preday_stats,columns_15d_by_preday)
        data.dropna(axis=0,inplace=True)
        res = res.append(data)
    return res

In [18]:
# 按航线分组，加快处理速度
def route_split(data):
    res = []
    df = data.copy()
    for i in df.label.unique():
        tmp = df[df.label == i]
        tmp.reset_index(inplace=True) 
        tmp.drop('index',inplace=True,axis=1)
        res.append(tmp)
    return res

In [19]:
###优化代码#########################################################################################################
# df1 = df.copy()
# df1 = df1[df1.label==0]
# df1 = route_split(df1)
# testdata=df1[0].copy().loc[df.dep_date>'2018-5-1']
# columns=columns_15d_by_dep
# print(df.head())
# 同一起飞日期，搜索日期的历史15天每天价格,
def history_15d_by_dep_optimize(data,columns):
    print()
    start=time.time()
    tmp=data.copy()
    # 给新增的列赋值空
    for col in columns:
        tmp[col]=np.nan
    #填充历史价格信息
    dep_dates=pd.to_datetime(tmp.dep_date.unique())
    for ts in dep_dates: 
        for i in range(len(columns)): #在相同出发日期的情况下进行位移，取得历史价格赋值
            tmp.loc[tmp.dep_date==ts,columns[i]]=tmp.loc[tmp.dep_date==ts,'minprice'].shift(1+i)               
    end=time.time()    
    print('特征构造-相同起飞日期，历史15天每天搜索价格特征耗时：%d秒'%((end-start)))
    return tmp
#测试调用函数（OK）
# %time aa= history_15d_by_dep_optimize(testdata,columns)
# aa.tail(60)
# 

In [17]:
##################################优化代码，填充相同提前天数历史搜索价格#############################3
start=time.time()
testdata=df.copy()
# testdata=df.copy().loc[df.dep_date>'2018-4-1']
columns=columns_15d_by_preday
def history_15d_by_preday_optimize(data,columns):
    start=time.time()
    tmp=data.copy()   
    # 给新增的列赋值空
    null_count=0
    for col in columns:
        tmp[col]=np.nan
    dep_dates=pd.to_datetime(tmp.dep_date.unique())
    for ts in dep_dates:
        for i in range(len(columns)):
            lagging = tmp.loc[(tmp.dep_date==ts-timedelta(i+1)),'minprice'] # 整列赋值
            if lagging.empty:
#                 print('do nothing')
                null_count+=1 
            else: #@ 将出发日期前一天的相同提前天数的数据填充到相应的历史价格字段                
                tmp.loc[(tmp.dep_date==ts),columns[i]]= lagging.values
    end=time.time()    
    print('特征构造-相同提前天数，历史15天每天搜索价格特征耗时：%d秒,填充空值：%s'%((end-start),null_count))
    return tmp
#调用函数（OK）
# aa= history_15d_by_preday_optimize(testdata,columns)
# aa.head()


# =====公共函数=====

### 2.统计量信息

1.统计量处理总函数

In [14]:
reg = re.compile('\s+|\t|\n') # 正则匹配空格、Tab键、换行
#列名同一出发日期历史15天每天票价
col_15d_by_dep='''history_1d_by_dep_price,history_2d_by_dep_price,history_3d_by_dep_price,history_4d_by_dep_price
    ,history_5d_by_dep_price,history_6d_by_dep_price,history_7d_by_dep_price,history_8d_by_dep_price
    ,history_9d_by_dep_price,history_10d_by_dep_price,history_11d_by_dep_price,history_12d_by_dep_price
    ,history_13d_by_dep_price,history_14d_by_dep_price,history_15d_by_dep_price'''
columns_15d_by_dep=re.sub(reg,'',col_15d_by_dep).split(',')    
#列名，同一提前天数历史15天每天票价
col_15d_by_preday='''history_1d_by_preday_price,history_2d_by_preday_price,history_3d_by_preday_price,
    history_4d_by_preday_price,history_5d_by_preday_price,history_6d_by_preday_price,
    history_7d_by_preday_price,history_8d_by_preday_price
    ,history_9d_by_preday_price,history_10d_by_preday_price,history_11d_by_preday_price,history_12d_by_preday_price
    ,history_13d_by_preday_price,history_14d_by_preday_price,history_15d_by_preday_price'''
columns_15d_by_preday=re.sub(reg,'',col_15d_by_preday).split(',')    
#列名，同一出发日期历史1周，2周统计量
col_by_dep_stats='''history_1w_by_dep_mean,history_1w_by_dep_var,history_1w_by_dep_min,history_1w_by_dep_max,
    history_1w_by_dep_median,history_1w_by_dep_mode,history_1w_by_dep_decline_count,
    history_1w_by_dep_rise_count,
    history_2w_by_dep_mean,history_2w_by_dep_var,history_2w_by_dep_min,history_2w_by_dep_max,
    history_2w_by_dep_median,history_2w_by_dep_mode,history_2w_by_dep_decline_count,
    history_2w_by_dep_rise_count'''
columns_by_dep_stats=re.sub(reg,'',col_by_dep_stats).split(',')

#列名，同一提前天数历史1周，2周统计量
col_by_preday_stats='''history_1w_by_preday_mean,history_1w_by_preday_var,history_1w_by_preday_min,history_1w_by_preday_max,
    history_1w_by_preday_median,history_1w_by_preday_mode,history_1w_by_preday_decline_count,
    history_1w_by_preday_rise_count,
    history_2w_by_preday_mean,history_2w_by_preday_var,history_2w_by_preday_min,history_2w_by_preday_max,
    history_2w_by_preday_median,history_2w_by_preday_mode,history_2w_by_preday_decline_count,
    history_2w_by_preday_rise_count'''
columns_by_preday_stats=re.sub(reg,'',col_by_preday_stats).split(',')  

#######统计指标总函数
def statistics_feature_engineering(data):
    df=data.copy()
    df=history_15d_by_dep_price_feature(df,columns_15d_by_dep)
    df=history_15d_by_preday_price_feature(df,columns_15d_by_preday)
    df=stats_by_dep_feature(df,columns_by_dep_stats,columns_15d_by_dep)
    df=stats_by_preday_feature(df,columns_by_preday_stats,columns_15d_by_preday)
    return df

In [27]:
##########################同一起飞日期，历史7天和15天价格统计指标
def stats_by_dep_feature(data,columns,names):
    start=time.time()
    df=data.copy() 
    for col in columns:
        df[col]=np.nan
    subset=df[names[0:7]].T
    df['history_1w_by_dep_mean']=subset.mean()
    df['history_1w_by_dep_var']=subset.var()
    df['history_1w_by_dep_min']=subset.min()
    df['history_1w_by_dep_max']=subset.max()
    df['history_1w_by_dep_median']=subset.median()
    mode=subset.mode().iloc[0]    
    df['history_1w_by_dep_mode']=mode
    dr=subset.apply(decline_and_rise_count)#获取上涨和下降次数
    df['history_1w_by_dep_decline_count']=dr.apply(lambda x:x[0])
    df['history_1w_by_dep_rise_count']=dr.apply(lambda x:x[1])    
#     df['history_1w_by_dep_mode_count']=mode.count()  # 暂时不考虑    
#     df['history_1w_by_dep_change_count']=    #暂时不考虑    
###########取15天的统计量
    subset=df[names].T
    df['history_2w_by_dep_mean']=subset.mean()
    df['history_2w_by_dep_var']=subset.var()
    df['history_2w_by_dep_min']=subset.min()
    df['history_2w_by_dep_max']=subset.max()
    df['history_2w_by_dep_median']=subset.median()
    mode=subset.mode().iloc[0]    
    df['history_2w_by_dep_mode']=mode
    dr=subset.apply(decline_and_rise_count)#获取上涨和下降次数
    df['history_2w_by_dep_decline_count']=dr.apply(lambda x:x[0])
    df['history_2w_by_dep_rise_count']=dr.apply(lambda x:x[1])    
    end=time.time()    
    print('特征构造--相同起飞日期价格统计量，耗时：%d秒'%((end-start))) 
    return df

##########################同一提前天数，历史7天和15天价格统计指标
def stats_by_preday_feature(data,columns,names):
    start=time.time()
    df=data.copy() 
    for col in columns:
        df[col]=np.nan
    subset=df[names[0:7]].T
    df['history_1w_by_preday_mean']=subset.mean()
    df['history_1w_by_preday_var']=subset.var()
    df['history_1w_by_preday_min']=subset.min()
    df['history_1w_by_preday_max']=subset.max()
    df['history_1w_by_preday_median']=subset.median()
    mode=subset.mode().iloc[0]    
    df['history_1w_by_preday_mode']=mode
    dr=subset.apply(decline_and_rise_count)#获取上涨和下降次数
    df['history_1w_by_preday_decline_count']=dr.apply(lambda x:x[0])
    df['history_1w_by_preday_rise_count']=dr.apply(lambda x:x[1])    
#     df['history_1w_by_dep_mode_count']=mode.count()  # 暂时不考虑    
#     df['history_1w_by_dep_change_count']=    #暂时不考虑    
###########取15天的统计量
    subset=df[names].T
    df['history_2w_by_preday_mean']=subset.mean()
    df['history_2w_by_preday_var']=subset.var()
    df['history_2w_by_preday_min']=subset.min()
    df['history_2w_by_preday_max']=subset.max()
    df['history_2w_by_preday_median']=subset.median()
    mode=subset.mode().iloc[0]    
    df['history_2w_by_preday_mode']=mode
    dr=subset.apply(decline_and_rise_count)#获取上涨和下降次数
    df['history_2w_by_preday_decline_count']=dr.apply(lambda x:x[0])
    df['history_2w_by_preday_rise_count']=dr.apply(lambda x:x[1])  
    end=time.time()    
    print('特征构造--相同起飞日期价格统计量，耗时：%d秒'%((end-start)))
 
    return df

#计算价格上升和下降次数
def decline_and_rise_count(data): 
    subset = list(data.copy())
    decline = 0
    rise = 0 
    modecount = 0
    for i in range(len(subset)-1):
        if subset[i]>subset[i+1]:
            decline += 1
        if subset[i]<subset[i+1]:
            rise += 1  
#     print(subset.count(mode))
    return decline,rise

### 1.季节性信息

In [16]:
def seasonal_feature_engineering(data):
    df=data.copy()
    df['dep_month_of_year']=df.dep_date.apply(lambda x : x.month) #月份
    df['dep_day_of_week']=df.dep_date.apply(lambda x : x.weekday()) #周几，从0开始
    df['dep_week_of_year']=df.dep_date.apply(lambda x : x.weekofyear) #一年第几周，从0开始
    df['search_month_of_year']=df.search_date.apply(lambda x : x.month) #月份
    df['search_day_of_week']=df.search_date.apply(lambda x : x.weekday()) #周几，从0开始
    df['search_week_of_year']=df.search_date.apply(lambda x : x.weekofyear) #一年第几周，从0开始   
    return df 
##测试调用 OK
# testdata=df.copy().loc[df.dep_date>'2018-5-1']
# aa=seasonal_feature_engineering(testdata)
# aa.head()